In [1]:
import os 

import fiona #must be import before geopandas
import geopandas as gpd
import rasterio
import rioxarray
import re
import rtree
import shapely
import pickle
import pyproj

from shapely.geometry import Polygon, Point
from shapely.ops import transform
#from cartopy import crs
import collections
import cv2
import math
from glob import glob
from tqdm.notebook import tqdm_notebook
import tqdm
# Standard packages
import tempfile
import warnings
import urllib
import shutil

# Less standard, but still pip- or conda-installable
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

from lxml.etree import Element,SubElement,tostring
import xml.dom.minidom
from xml.dom.minidom import parseString
import xml.etree.ElementTree as et
from xml.dom import minidom

#import requests
from PIL import Image
from io import BytesIO
import tqdm
from skimage.metrics import structural_similarity as compare_ssim
import imutils

import psutil

import data_eng.az_proc as ap
import data_eng.form_calcs as fc


#Parsing/Modifying XML
from lxml.etree import Element,SubElement,tostring
import xml.dom.minidom
from xml.dom.minidom import parseString
import xml.etree.ElementTree as et
from xml.dom import minidom

## File Paths

In [10]:
parent_directory = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//"

#create folder to hold tiles in completed dataset
#load tiles and tile xmls
tiles_dir = os.path.join(parent_directory,"complete_dataset","tiles")
os.makedirs(tiles_dir, exist_ok=True)

tiles_xml_dir = os.path.join(parent_directory,"complete_dataset","tiles_xml")
os.makedirs(tiles_xml_dir, exist_ok=True)
tiles_xml_list = os.listdir(tiles_xml_dir)

#NAIP quad map
quad_indicies_path = "C:/Users/rapiduser/Box/EPA STAR 2019 (Community Resistance to Environmental Disasters)/Data/AST Datasets/MapIndices_National_GDB/MapIndices_National_GDB.gdb"
#fiona.listlayers(quad_indicies_path)
rechipped_image_path = os.path.join(parent_directory,"temp/rechip")

folders_of_images_xmls_by_tile = os.listdir(rechipped_image_path)
images_and_xmls_by_tile_path = rechipped_image_path

tile_name_tile_url_complete_array_path = "image_download_azure/tile_name_tile_url_complete_array.csv"
tile_name_tile_url_complete_array = pd.read_csv(tile_name_tile_url_complete_array_path)
tile_name_tile_url_complete_array = tile_name_tile_url_complete_array.to_numpy()
tile_name_tile_url_complete_array = tile_name_tile_url_complete_array[:,(1,2)] #remove index

# Add chips to rechip folder
Add all chips corresponding to a tile to the 'chips' folder for each tile

In [3]:
#for tile_name in tqdm.tqdm(folders_of_images_xmls_by_tile):
#    add_chips_to_chip_folders(rechipped_image_path, tile_name)

# Correct object labels

## Fix inconsistent labels 

In [ ]:
for tile_name in tqdm.tqdm(folders_of_images_xmls_by_tile):
    #specify image/xml paths for each tile
    positive_xml_dir = os.path.join(images_and_xmls_by_tile_path, tile_name, "chips_positive_xml")
    fc.correct_inconsistent_labels_xml(positive_xml_dir)

## Reclassify narrow closed roof tanks and closed roof tanks by size

In [ ]:
for tile in tqdm.tqdm(folders_of_images_xmls_by_tile): #iterate over tile folders
    tile_annotation_path = os.path.join(images_and_xmls_by_tile_path, tile, "chips_positive_xml")
    for xml in os.listdir(tile_annotation_path):
        xml_path = os.path.join(tile_annotation_path, xml)
        fc.reclassify_narrow_closed_roof_and_closed_roof_tanks(xml_path)

# Generate table of characteristics for tiles/images

In [5]:
tile_characteristics, image_characteristics = fc.image_tile_characteristics(rechipped_image_path, tiles_dir)#, tile_name_tile_url_complete_array)

100%|██████████████████████████████████████████████████████████████████████████████| 2139/2139 [25:57<00:00,  1.37it/s]


In [ ]:
tile_dimensions_and_utm_coords()

In [39]:
image_characteristics = pd.read_csv("image_characteristics.csv")
tile_characteristics = pd.read_csv("tile_characteristics.csv")

# Create tile level annotations

## Generate tile level XMLs

In [4]:
#fc.generate_tile_xmls(rechipped_image_path, tiles_dir, tiles_xml_dir, int(512))

## Merge neighboring bounding boxes within each tile
References:
https://answers.opencv.org/question/231263/merging-nearby-rectanglesedited/
https://stackoverflow.com/questions/55593506/merge-the-bounding-boxes-near-by-into-one

In [51]:
distance_limit = 5
tile_database = fc.merge_tile_annotations(tile_characteristics, tiles_xml_dir, distance_limit = 5)
#merged_tile_database =  tile_database[tile_database["merged_bbox"] == True]

100%|██████████████████████████████████████████████████████████████████████████████| 2120/2120 [58:26<00:00,  1.65s/it]


# Clip storm surge data and append to tile level annotation database

#sc_slosh_inundation_map_path = "C:/Users/rapiduser/Box/EPA STAR 2019 (Community Resistance to Environmental Disasters)/Data/Natural Hazards Data/NHC Potential Storm Surge Flood Map/SC_SLOSH_MOM_Inundation"
#tile_database = fc.identify_inundation_for_tanks(tile_database, sc_slosh_inundation_map_path)

# Add in state

In [ ]:
states_gpds_path = "C:/Users/rapiduser/Box/EPA STAR 2019 (Community Resistance to Environmental Disasters)/Data/US Political Boundaries/US State Boundaries/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
tile_database = fc.identify_state_name_for_each_state(states_gpds_path, tile_database)
#check issues in state list
#state_list = state_list[state_list!=None]
#np.unique(state_list)

In [34]:
np.unique(df["object_cla"])

array(['closed_roof_tank', 'external_floating_roof_tank',
       'narrow_closed_roof_tank', 'sedimentation_tank', 'spherical_tank',
       'undefined_object', 'water_tower'], dtype=object)

In [38]:
tile_level_annotations

NameError: name 'tile_level_annotations' is not defined

In [16]:
#https://localcoder.org/merge-the-bounding-boxes-near-by-into-one

# Save tile dabasebase

In [18]:
output_filepath = os.path.join(parent_directory,"complete_dataset","tile_level_annotations")
output_filename = 'tile_level_annotations'
fc.write_gdf(tile_database, output_filepath, output_filename)